In [ ]:
prefix = 'kmeans'

import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
import matplotlib.pyplot as plt
df = pd.read_csv('training-data/kmeans_input.csv')
x = df.iloc[:,0].values
y = df.iloc[:,1].values
plt.scatter(x,y)

In [ ]:
WORK_DIRECTORY = 'training-data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

estimator = sage.estimator.Estimator(
    image, 
    role, 
    1, 
    'ml.c4.2xlarge', 
    output_path="s3://{}/{}/output".format(sess.default_bucket(), prefix),
    sagemaker_session=sess)

estimator.fit(data_location)

In [ ]:
# DOWNLOAD MODEL AND SHOW CLASSES FOUND

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = estimator.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

In [ ]:
input = """x1,x2
-2,-5
2,10
0,0"""
r = predictor.predict(input).decode('utf-8')
print(r)